In [1]:
%load_ext rpy2.ipython

In [2]:
%R require(gender)
%R require(genderdata)
%R require(tibble)

R[write to console]: Loading required package: gender

R[write to console]: Loading required package: genderdata

R[write to console]: Loading required package: tibble



1


In [9]:
%matplotlib notebook

import pandas as pd
import numpy as np

import time
import pickle

import memory_profiler

%load_ext memory_profiler

from pathlib import Path

%load_ext watermark

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [10]:
import black
import jupyter_black

jupyter_black.load(line_length=79)

In [11]:
variables_path = Path("../results/variables/iclr24v2a")
figures_path = Path("../results/figures")
data_path = Path("../data")

In [12]:
%watermark -a 'Rita González-Márquez' -t -d -tz -u -v -iv -w -m -h -p transformers,openTSNE
print(distro.name(pretty=True))

Author: Rita González-Márquez

Last updated: 2024-03-27 03:40:37CET

Python implementation: CPython
Python version       : 3.8.10
IPython version      : 8.4.0

transformers: 4.30.2
openTSNE    : 0.6.2

Compiler    : GCC 9.4.0
OS          : Linux
Release     : 3.10.0-1160.el7.x86_64
Machine     : x86_64
Processor   : x86_64
CPU cores   : 96
Architecture: 64bit

Hostname: 94343264f94c

numpy          : 1.23.1
black          : 23.1.0
jupyter_black  : 0.3.3
memory_profiler: 0.60.0
watermark      : 2.3.1
pandas         : 1.4.3

Watermark: 2.3.1



NameError: name 'distro' is not defined

# Import

In [13]:
%%time
iclr2024 = pd.read_parquet(
    data_path / "iclr24v2a.parquet",
    engine="pyarrow",
)

CPU times: user 292 ms, sys: 122 ms, total: 414 ms
Wall time: 424 ms


In [14]:
iclr2024.keywords = iclr2024.keywords.transform(lambda x: list(x))
iclr2024.scores = iclr2024.scores.transform(lambda x: list(x))

In [15]:
iclr2024

,year,id,title,abstract,authors,decision,scores,keywords,labels
0,2017,B1-Hhnslg,Prototypical Networks for Few-shot Learning,A recent approach to few-shot classification c...,"Jake Snell, Kevin Swersky, Richard Zemel",Reject,"[6, 4, 5]","[deep learning, transfer learning]",transfer learning
1,2017,B1-q5Pqxl,Machine Comprehension Using Match-LSTM and Ans...,Machine comprehension of text is an important ...,"Shuohang Wang, Jing Jiang",Accept (Poster),"[6, 6, 7]","[natural language processing, deep learning]",language models
2,2017,B16Jem9xe,Learning in Implicit Generative Models,Generative adversarial networks (GANs) provide...,"Shakir Mohamed, Balaji Lakshminarayanan",Invite to Workshop Track,"[8, 7, 6]",[unsupervised learning],unlabeled
3,2017,B16dGcqlx,Third Person Imitation Learning,Reinforcement learning (RL) makes it possible ...,"Bradly C Stadie, Pieter Abbeel, Ilya Sutskever",Accept (Poster),"[6, 5, 6]",[],unlabeled
4,2017,B184E5qee,Improving Neural Language Models with a Contin...,We propose an extension to neural network lang...,"Edouard Grave, Armand Joulin, Nicolas Usunier",Accept (Poster),"[7, 9, 5]",[natural language processing],language models
...,...,...,...,...,...,...,...,...,...
24440,2024,zxPDdw8koz,CLIP meets Model Zoo Experts: Pseudo-Supervisi...,Contrastive language image pretraining (CLIP) ...,"Mohammadreza Salehi, Mehrdad Farajtabar, Maxwe...",Withdrawn,"[8, 3, 3, 3]","[contrastive learning, clip, distillation, den...",vision-language models
24441,2024,zyBJodMrn5,On the generalization capacity of neural netwo...,The advent of the Transformer has led to the d...,"Takuya Ito, Soham Dan, Mattia Rigotti, James K...",Accept (poster),"[8, 3, 6]","[compositional generalization, compositionalit...",out-of-distribution
24442,2024,zz61V8bIab,Stochastic Adversarial Networks for Multi-Doma...,Adversarial training has played a pivotal role...,"Xu Wang, Yuan Wu",Withdrawn,"[5, 1, 5]","[multi-domain text classification, adversarial...",adversarial
24443,2024,zzqn5G9fjn,Breaking Physical and Linguistic Borders: Mult...,Pretrained large language models (LLMs) have e...,"Wanru Zhao, Royson Lee, Yihong Chen, Xinchi Qi...",Accept (poster),"[5, 3, 1, 8]","[multilingual federated learning, natural lang...",language models


## Split names

In [16]:
authors = iclr2024.authors.to_list()
authors[:10]

['Jake Snell, Kevin Swersky, Richard Zemel',
 'Shuohang Wang, Jing Jiang',
 'Shakir Mohamed, Balaji Lakshminarayanan',
 'Bradly C Stadie, Pieter Abbeel, Ilya Sutskever',
 'Edouard Grave, Armand Joulin, Nicolas Usunier',
 'Levent Sagun, Leon Bottou, Yann LeCun',
 'Jianwen Xie, Yang Lu, Ruiqi Gao, Song-Chun Zhu, Ying Nian Wu',
 'Vincent Dumoulin, Ishmael Belghazi, Ben Poole, Alex Lamb, Martin Arjovsky, Olivier Mastropietro, Aaron Courville',
 'Joji Toyama, Masanori Misono, Masahiro Suzuki, Kotaro Nakayama, Yutaka Matsuo',
 'Sahil Sharma, Aravind S. Lakshminarayanan, Balaraman Ravindran']

In [17]:
%%time
first_author_first_name = [
    elem.split(",")[0].strip().split(" ")[0] for elem in authors
]

CPU times: user 15.1 ms, sys: 3.26 ms, total: 18.4 ms
Wall time: 17 ms


In [18]:
%%time
last_author_first_name = [
    elem.split(",")[-1].strip().split(" ")[0] for elem in authors
]

CPU times: user 17.2 ms, sys: 337 µs, total: 17.5 ms
Wall time: 17 ms


### Exploration

In [19]:
print(np.sum(np.array(first_author_first_name) == ""))
print(np.sum(np.array(last_author_first_name) == ""))

0
0


# Gender prediction

##  First author

In [20]:
dict_names = {"Names": first_author_first_name}
df = pd.DataFrame(dict_names)
df_names_first_author = df.Names
df_names_first_author

0                Jake
1            Shuohang
2              Shakir
3              Bradly
4             Edouard
             ...     
24440    Mohammadreza
24441          Takuya
24442              Xu
24443           Wanru
24444       Chunghyun
Name: Names, Length: 24445, dtype: object

In [21]:
%%time
%R library(gender)
%R -i df_names_first_author -o result result = gender(df_names_first_author, years = 2012, method = "ssa")

CPU times: user 1.22 s, sys: 81.6 ms, total: 1.3 s
Wall time: 1.28 s


,name,proportion_male,proportion_female,gender,year_min,year_max
1,Aadarsh,1.0000,0.0000,male,2012.0,2012.0
2,Aakash,1.0000,0.0000,male,2012.0,2012.0
3,Aaron,0.9970,0.0030,male,2012.0,2012.0
4,Aaron,0.9970,0.0030,male,2012.0,2012.0
5,Aaron,0.9970,0.0030,male,2012.0,2012.0
...,...,...,...,...,...,...
10212,Zoe,0.0023,0.9977,female,2012.0,2012.0
10213,Zoe,0.0023,0.9977,female,2012.0,2012.0
10214,Zohar,0.5149,0.4851,male,2012.0,2012.0
10215,Zohar,0.5149,0.4851,male,2012.0,2012.0


In [22]:
gender_map = dict(zip(result.name, result.gender))

In [23]:
first_author_gender = np.vectorize(gender_map.get)(first_author_first_name)

##  Last author

In [24]:
dict_names = {"Names": last_author_first_name}
df = pd.DataFrame(dict_names)
df_names_last_author = df.Names
df_names_last_author

0         Richard
1            Jing
2          Balaji
3            Ilya
4         Nicolas
           ...   
24440      Sachin
24441      Murray
24442        Yuan
24443    Nicholas
24444       Minsu
Name: Names, Length: 24445, dtype: object

In [25]:
%%time
%R library(gender)
%R -i df_names_last_author -o result result = gender(df_names_last_author, years = 2012, method = "ssa")

CPU times: user 580 ms, sys: 14.5 ms, total: 594 ms
Wall time: 585 ms


,name,proportion_male,proportion_female,gender,year_min,year_max
1,Aaditya,1.0000,0.0000,male,2012.0,2012.0
2,Aaditya,1.0000,0.0000,male,2012.0,2012.0
3,Aaro,1.0000,0.0000,male,2012.0,2012.0
4,Aaron,0.9970,0.0030,male,2012.0,2012.0
5,Aaron,0.9970,0.0030,male,2012.0,2012.0
...,...,...,...,...,...,...
12203,Zohar,0.5149,0.4851,male,2012.0,2012.0
12204,Zohar,0.5149,0.4851,male,2012.0,2012.0
12205,Zoran,1.0000,0.0000,male,2012.0,2012.0
12206,Zoran,1.0000,0.0000,male,2012.0,2012.0


In [26]:
gender_map = dict(zip(result.name, result.gender))

In [27]:
last_author_gender = np.vectorize(gender_map.get)(last_author_first_name)

In [28]:
last_author_gender

array(['male', 'female', 'None', ..., 'female', 'male', 'None'],
      dtype='<U6')

## Exploration

In [29]:
print(np.sum(first_author_gender == "male"))
print(np.sum(first_author_gender == "female"))
print(np.sum(first_author_gender == "None"))

8449
1767
14229


In [30]:
print(
    np.sum(first_author_gender == "male")
    + np.sum(first_author_gender == "female")
    + np.sum(first_author_gender == "None")
)
first_author_gender.shape

24445


(24445,)

In [31]:
print(np.sum(last_author_gender == "male"))
print(np.sum(last_author_gender == "female"))
print(np.sum(last_author_gender == "None"))

10419
1788
12238


In [32]:
print(
    np.sum(last_author_gender == "male")
    + np.sum(last_author_gender == "female")
    + np.sum(last_author_gender == "None")
)

24445


# Save

In [33]:
dict_names = {
    "first_author_name": first_author_first_name,
    "first_author_gender": first_author_gender,
    "last_author_name": last_author_first_name,
    "last_author_gender": last_author_gender,
}
df = pd.DataFrame(dict_names)
df

,first_author_name,first_author_gender,last_author_name,last_author_gender
0,Jake,male,Richard,male
1,Shuohang,None,Jing,female
2,Shakir,male,Balaji,None
3,Bradly,male,Ilya,male
4,Edouard,male,Nicolas,male
...,...,...,...,...
24440,Mohammadreza,None,Sachin,male
24441,Takuya,None,Murray,male
24442,Xu,None,Yuan,female
24443,Wanru,None,Nicholas,male


In [34]:
# save
df.to_parquet(
    variables_path / "names_and_genders.parquet",
    index=False,
    engine="pyarrow",
)